# Word Embeddings : le modèle Word2Vec

## Imports

In [14]:
import numpy as np
import pandas as pd
from pathlib import Path
import re
import nltk
import string

from nltk.corpus import stopwords
from nltk import word_tokenize

from gensim.models import Word2Vec
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to C:\Users\Ing Armel
[nltk_data]     Fopa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Ing Armel
[nltk_data]     Fopa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to C:\Users\Ing Armel
[nltk_data]     Fopa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Liste des fichiers CAMille

In [15]:
from pathlib import Path

# Dossier contenant les fichiers CAMille
data_dir = Path("../../data/txt")

print("Dossier existe ? ", data_dir.exists())

all_txt = sorted(data_dir.glob("*.txt"))
print("Nombre total de fichiers texte :", len(all_txt))

# Aperçu des 10 premiers fichiers
for p in all_txt[:10]:
    print(" -", p.name)

Dossier existe ?  True
Nombre total de fichiers texte : 51
 - KB_JB230_1892-08-07_01-0003.txt
 - KB_JB230_1903-10-16_01-0002.txt
 - KB_JB230_1913-07-05_01-0001.txt
 - KB_JB258_1884-09-03_01-0003.txt
 - KB_JB258_1894-12-09_01-0003.txt
 - KB_JB258_1906-01-09_01-0002.txt
 - KB_JB421_1899-05-15_01-00003.txt
 - KB_JB421_1926-10-29_01-00002.txt
 - KB_JB421_1950-04-15_01-00004.txt
 - KB_JB427_1920-01-10_01-00004.txt


Selction des fichiers de la decennie 1920

In [16]:

# Sélection de la décennie
DECADE = "192"
pattern = re.compile(rf"{DECADE}\d")

# Fichiers correspondant à la décennie choisie
files_decade = [p for p in all_txt if pattern.search(p.name)]
print(f"Nombre de fichiers pour la décennie {DECADE}0 :", len(files_decade))
for p in files_decade:
    print(" -", p.name)

Nombre de fichiers pour la décennie 1920 : 7
 - KB_JB421_1926-10-29_01-00002.txt
 - KB_JB427_1920-01-10_01-00004.txt
 - KB_JB494_1922-09-28_01-0005.txt
 - KB_JB567_1924-08-30_01-00003.txt
 - KB_JB572_1927-07-20_01-00005.txt
 - KB_JB729_1927-11-15_01-00004.txt
 - KB_JB837_1925-01-01_01-00003.txt


### preparation et tokenisation du corpus

Preparation et nettoyage du texte

In [17]:
from nltk.corpus import stopwords
import string

# Stopwords français
sw = set(stopwords.words("french"))

def nettoyer_texte(texte):
    """Nettoyage léger : mise en minuscule, suppression des chiffres et ponctuation."""
    texte = texte.lower()
    texte = re.sub(r"\d+", " ", texte)                # supprime les chiffres
    texte = texte.translate(str.maketrans("", "", string.punctuation))  # supprime la ponctuation
    return texte

Tokenisation du corpus

In [18]:

from nltk.tokenize import word_tokenize

sentences = []

for path in files_decade:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        txt = nettoyer_texte(f.read())
        tokens = [w for w in word_tokenize(txt) if w not in sw and len(w) > 2]
        sentences.append(tokens)

print(f"Nombre de documents préparés : {len(sentences)}")
print("Exemple de tokens :", sentences[0][:25])

Nombre de documents préparés : 7
Exemple de tokens : ['mariage', 'prince', 'léopold', 'départ', 'famille', 'royale', 'famille', 'royale', 'rendant', 'stockholm', 'assister', 'mariage', 'prince', 'léopold', 'princesse', 'princesse', 'astrid', 'lâge', 'huit', 'ans', 'astrid', 'quittera', 'bruxelles', 'samedi', 'scir']


## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [ ]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [ ]:
type(bigram_phrases.vocab)

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [ ]:
len(bigram_phrases.vocab.keys())

Prenons une clé au hasard :

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

Le dictionnaire indique le score de cette coocurrence :

In [ ]:
bigram_phrases.vocab[key_]

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

## Entrainement d'un modèle Word2Vec sur ce corpus

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [ ]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [ ]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [ ]:
model.wv["ministre"]

### Calculer la similarité entre deux termes

In [ ]:
model.wv.similarity("ministre", "roi")

### Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("ministre", topn=10)

### Faire des recherches complexes à travers l'espace vectoriel

In [ ]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))